## Import libraries

In [168]:
import ee
import geemap
import os, re, json, time
import geopandas as gpd
from datetime import datetime, timedelta
from shapely.geometry import Polygon, mapping

## Start Connection with Google Earth Engine

In [ ]:
ee.Authenticate(force=True)
ee.Initialize()
print(ee.String('Hello from the Earth Engine servers!').getInfo())

## Set up environmental variables

In [49]:
root_folder = os.getcwd().split('\\')
root_folder = root_folder[1:-2]
if root_folder[-1] != 'fireRunSeverity':
    print("Didn't get correct root folder!")
    print(root_folder)
root_folder = os.path.join('C:\\', *root_folder)

in_Name = 'GIF14_Au'
root_folder


'C:\\Users\\user\\OneDrive - University of Eastern Finland\\CourseLleida\\_Master Thesis\\fireRunSeverity'

## Load EE side data

In [6]:
# Add Earth Engine dataset
S2_harmon = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
Map = geemap.Map(center=[40, -100], zoom=4)
Map.setCenter(148.3462, -37.3684,9)


## Load Local side data

In [87]:
dataFLD = os.path.join(root_folder, 'data', 'fireruns', in_Name)
inFLD = os.path.join(dataFLD, 'input')
shpIn  = []
for f in os.scandir(inFLD):
    if re.search(r".shp$", f.name):
        print(f.name)
        shpIn.append(f)

if len(shpIn) != 1:
    print("Check No. of shp file not equal to 1!")
else:
    shpIn = shpIn[0]

# Read Shp file
shpGPD = gpd.read_file(os.path.join(inFLD, shpIn))
print(shpGPD.crs)
shp_reproj = shpGPD.to_crs("EPSG:4326")
# Get the bounds
minx, miny, maxx, maxy = shp_reproj.total_bounds
# Create a polygon of bounds
bounds_polygon = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny), (minx, miny)])
# Convert the polygon to GeoJSON
json_polygon = mapping(bounds_polygon)
print(json_polygon)

GIF14_32755_multipart.shp
EPSG:32755
{'type': 'Polygon', 'coordinates': (((148.797548490504, -37.78877807241547), (148.797548490504, -37.04608874283253), (150.0295287036503, -37.04608874283253), (150.0295287036503, -37.78877807241547), (148.797548490504, -37.78877807241547)),)}


In [133]:
# Get date of data
fire_dtstring = list(shpGPD['FeHo'])
print(fire_dtstring)
fire_dtsList  = [datetime.strptime(dt, "%Y/%m/%d_%H%M") for dt in fire_dtstring]
print(sorted(fire_dtsList))
dateSt = min(fire_dtsList).date()
dateEd = max(fire_dtsList).date()
print("Fire start date and end date:")
print(dateSt, dateEd)


# Compute the date period for retrieving Satellite img
prefire_date  = [dateSt - timedelta(days = 4*30), dateSt - timedelta(days = 1)]
postfire_date = [dateEd + timedelta(days = 1),    dateEd + timedelta(days = 4*30)]

prefire_date = [d.strftime("%Y-%m-%d") for d in prefire_date]
postfire_date = [d.strftime("%Y-%m-%d") for d in postfire_date]

print("Pre- and Post- fire periods selected for satellite images:")
print(prefire_date, postfire_date)

['2019/12/30_1400', '2019/12/30_0100', '2020/1/01_0100', '2020/1/02_1400', '2020/1/03_1400', '2020/1/01_1400', '2020/1/01_1400', '2020/1/02_0100', '2020/1/02_0100', '2020/1/01_1400', '2020/1/01_1400', '2020/1/02_1400', '2020/1/03_1400', '2020/1/03_1400', '2020/1/01_0100', '2020/1/03_1400', '2020/1/01_1400', '2020/1/02_1400', '2020/1/02_1400', '2020/1/01_0100', '2020/1/03_0100', '2020/1/02_1400', '2020/1/01_1400', '2020/1/04_1400', '2020/1/04_0100', '2020/1/05_0100', '2020/1/12_0100', '2020/1/02_0100', '2020/1/01_1400', '2020/1/04_0100', '2020/1/04_1400', '2020/1/05_0100', '2020/1/12_1400', '2020/1/12_1400', '2020/1/04_1400', '2020/1/15_0100', '2020/1/15_0100', '2020/1/13_1400', '2020/1/13_0100', '2020/1/12_1400', '2020/1/13_1400', '2020/1/13_1400', '2020/1/11_1400', '2020/1/11_1400', '2020/1/11_1400', '2020/1/13_0100', '2020/1/14_0100', '2020/1/11_1400', '2020/1/15_0100', '2020/1/12_1400', '2020/1/13_0100', '2020/1/13_1400', '2020/1/13_0100', '2020/1/13_0100', '2020/1/14_0100', '2020/1

## Upload data to EE

In [93]:
clipAOI = ee.Geometry.Polygon(json_polygon["coordinates"])
clipAOI2 = clipAOI.buffer(50000)
print(clipAOI.getInfo())

{'type': 'Polygon', 'coordinates': [[[148.797548490504, -37.78877807241547], [150.0295287036503, -37.78877807241547], [150.0295287036503, -37.04608874283253], [148.797548490504, -37.04608874283253], [148.797548490504, -37.78877807241547]]]}


In [145]:
Map.addLayer(clipAOI2, {'color': "blue"}, "Input SHP range")

## (GEE) Process satellite

In [135]:
# filter satellite data
bandList = ["B.", "B..", "QA60", "MSK_CLDPRB"]
S2_pre_select = S2_harmon.filterDate(prefire_date[0], prefire_date[1]) \
.filterBounds(clipAOI2).select(bandList)
# .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))

S2_pos_select = S2_harmon.filterDate(postfire_date[0], postfire_date[1]) \
.filterBounds(clipAOI2).select(bandList)
# .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))

In [146]:
# Functions for Computing Necessary indices
def func_maskClouds(image):
    imgCLP = image.select("MSK_CLDPRB")
    mskCLP = imgCLP.lt(10);  
    qa = image.select("QA60")
    Clouds = qa.bitwiseAnd(1 << 10).eq(0);  
    Cirrus = qa.bitwiseAnd(1 << 11).eq(0); 
    mask = Clouds.And(Cirrus).And(mskCLP)
    return image.updateMask(mask)

def func_rescale(image):
    refl = image.multiply(0.0001)
    return refl.copyProperties(image, ["system:time_start"])

def func_calcIndices (image):
    NBR = image.normalizedDifference(["B8", "B12"]).rename("NBR")
    NDVI = image.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return image.addBands([NBR, NDVI])




In [140]:
S2pre = S2_pre_select.map(func_maskClouds).map(func_rescale) \
.sort('system:time_start') \
.mosaic(); 

S2pos = S2_pos_select.map(func_maskClouds).map(func_rescale) \
.sort('system:time_start', False) \
.mosaic(); 
# print(json.dumps(S2pre.getInfo(), indent=4))


In [144]:
# Visualization
visP = {'bands': ["B4",  "B3",  "B2"], 'max': 0.3}
Map.remove(clipAOI2)
Map.addLayer(S2pre.clip(clipAOI), visP, "S2 pre fire")
Map.addLayer(S2pos.clip(clipAOI), visP, "S2 after fire")

In [147]:
# Indices Calculation
S2pre_id = func_calcIndices(S2pre)
S2pos_id = func_calcIndices(S2pos)
S2_dNBR  = S2pre_id.select("NBR").subtract(S2pos_id.select("NBR")).multiply(1000).rename("dNBR")
print("dNBR",S2_dNBR.getInfo())

dNBR {'type': 'Image', 'bands': [{'id': 'dNBR', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -2000, 'max': 2000}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [155]:
S2_dNBR.bandNames()

## Output image to Google drive folder

In [180]:
# Define export parameters
driveFLD = os.path.join('EarthEngineFolder', in_Name, in_Name)
tStamp = datetime.now().strftime("%d%m%Y_%H%M")
export_task = ee.batch.Export.image.toDrive(
    image=S2_dNBR,
    description='sentinel2_dNBR_image_export',
    folder='EarthEngineFolder',  # Folder in your Google Drive
    fileNamePrefix=in_Name+'---S2_dNBR---'+tStamp,  # File name prefix
    region=clipAOI,  # Define the region to export (could be an area of interest)
    scale=800,  # Spatial resolution in meters
    fileFormat='GeoTIFF',  # File format
    maxPixels=1e12  # Max pixels to export
)



# Start the export task
try:
    export_task.start()
    print("Exporting to Google Drive...")

    # Check the status of the export task
    while export_task.active():
        print('Exporting... Please wait.')
        time.sleep(10)

    print('Export completed.')

except ee.ee_exception.EEException as e:
    print(f"Error during export: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")


Exporting to Google Drive...
Exporting... Please wait.
Exporting... Please wait.
Export completed.


## Miscellaneous

#### Miscellaneous: Visualize calculated indices

In [149]:
# Produce Severity classes
S2_severity = ee.Image(1) \
.where(S2_dNBR.gt(-500).And(S2_dNBR.lte(-251)), 1) \
.where(S2_dNBR.gt(-251).And(S2_dNBR.lte(-101)), 2) \
.where(S2_dNBR.gt(-101).And(S2_dNBR.lte(99)), 3) \
.where(S2_dNBR.gt(99).And(S2_dNBR.lte(269)), 4) \
.where(S2_dNBR.gt(269).And(S2_dNBR.lte(439)), 5) \
.where(S2_dNBR.gt(439).And(S2_dNBR.lte(659)), 6) \
.where(S2_dNBR.gt(660).And(S2_dNBR.lte(1300)), 7) \
.clip(clipAOI)
# # ------------------------------------------------ # #



# # ------------------------------------------------ # #
# # NBR fire severity color:https:#un-spider.Org/advisory-support/recommended-practices/recommended-practice-burn-severity/Step-By-Step/RStudio
dnbrPal = ["556B2F","6E8B3D","32CD32", "EEEE00", "EE7600", "FF0000", "A020F0"]
# # NDVI palette https:#custom-scripts.sentinel-hub.com/sentinel-2/ndvi/
ndviPal = ["0c0c0c", "eaeaea", "ccc682", "91bf51", "70a33f", "4f892d", "306d1c", "0f540a", "004400"]
# # name of the legend
dnbrNames = ["Enhanced Regrowth, High", "Enhanced Regrowth, Low", "Unburned", "Low Severity", "Moderate-low Severity", "Moderate-high Severity", "High Severity"]

visNBR = {'bands': "NBR", 'max':1, 'min': -1, 'palette': ["green", "red"]}

visNDVI = {
    'bands': "NDVI",
    'max': 1,
    'min': -1,
    'palette': ndviPal
}
visdNBR = {
    'max': 1300,
    'min': -500,
    'palette': dnbrPal
}
visSevere = {
    'max': 7,
    'min': 1,
    'palette': dnbrPal
}

# m.addLayer(S2pre_id, visNDVI, "NDVI_pre")
# m.addLayer(S2pos_id, visNDVI, "NDVI_pos")
Map.addLayer(S2_dNBR.clip(clipAOI), visdNBR, "dNBR")
Map.addLayer(S2_severity.clip(clipAOI), visSevere, "dNBR Severity")
# # ------------------------------------------------ # #

Map




Map(bottom=80508.0, center=[-37.335224359306395, 509.32766905173236], controls=(WidgetControl(options=['positi…

#### Miscellaneous: Counting pixels

In [164]:
pixel_count = S2_dNBR.select('dNBR').reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=clipAOI,  # Use the image's geometry as the region
    scale=10,  # Spatial resolution in meters
    maxPixels=1e10  # Maximum number of pixels to process
)

# Print the pixel count
print('Pixel count for B4 band:', pixel_count.getInfo())

Pixel count for B4 band: {'dNBR': 113379278}
